# Research 02 — BTC Single Asset
## Trend + Mining with conditional OU overlay

This notebook keeps **Trend + Mining** as the anchor model and only adds an **OU sleeve** if it improves **net Sharpe** after fees/slippage.

It also adds:
- marginal value-add analysis,
- portfolio-fit diagnostics,
- enhanced visualization (including 3D Sharpe surface).

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

project_root = os.path.abspath("..")
if not os.path.exists(os.path.join(project_root, "config.py")):
    project_root = os.path.abspath("Crypto")
if project_root not in sys.path:
    sys.path.append(project_root)

from config import (
    DATA_PATH,
    DATA_START_DATE,
    FEE_BPS,
    SLIPPAGE_BPS,
    LEVERAGE_CAP,
    TREND_FAST_WINDOW,
    TREND_SLOW_WINDOW,
    TREND_LONG_ONLY,
    TREND_AGGRESSIVE,
    TREND_NEUTRAL,
    TREND_DEFENSIVE,
    MINING_Z_WINDOW,
    MINING_ENTRY_Z,
    MINING_EXIT_Z,
    MINING_USE_LOG_EDGE,
    OU_WINDOW,
    OU_ENTRY_Z,
    OU_EXIT_Z,
    OU_LONG_SHORT,
)
from Data.raw_data_loader import load_raw_crypto_csv
from Models.trend import trend_signal
from Models.mining import mining_signal
from Models.ou import ou_signal
from Backtest.engine import run_backtest
from Backtest.metrics import rolling_sharpe

DAYS_PER_YEAR = 365
plt.style.use("seaborn-v0_8-darkgrid")

In [ ]:
def sharpe(r: pd.Series) -> float:
    r = r.dropna()
    if len(r) < 5 or r.std(ddof=1) == 0:
        return np.nan
    return float(np.sqrt(DAYS_PER_YEAR) * r.mean() / r.std(ddof=1))


def annual_return(r: pd.Series) -> float:
    return float(r.dropna().mean() * DAYS_PER_YEAR)


def annual_vol(r: pd.Series) -> float:
    return float(r.dropna().std(ddof=1) * np.sqrt(DAYS_PER_YEAR))


def max_drawdown(eq: pd.Series) -> float:
    peak = eq.cummax()
    return float((eq / peak - 1.0).min())


def info_ratio(active_returns: pd.Series) -> float:
    ar = active_returns.dropna()
    if len(ar) < 5 or ar.std(ddof=1) == 0:
        return np.nan
    return float(np.sqrt(DAYS_PER_YEAR) * ar.mean() / ar.std(ddof=1))


def run_strategy(price: pd.Series, pos: pd.Series) -> dict:
    return run_backtest(
        price_series=price,
        position=pos,
        fee_bps=FEE_BPS,
        slippage_bps=SLIPPAGE_BPS,
        leverage_cap=LEVERAGE_CAP,
    )


def combine_positions(weights: dict, pos_map: dict, cap: float = LEVERAGE_CAP) -> pd.Series:
    combined = sum(weights[k] * pos_map[k] for k in weights)
    return combined.clip(-cap, cap)

In [ ]:
# Data load / clean

df = load_raw_crypto_csv(DATA_PATH, start_date=DATA_START_DATE)
df = df[(df["BTC-USD_close"] > 0) & (df["COST_TO_MINE"] > 0)].copy()
btc_ret = df["BTC-USD_close"].pct_change()
df = df[btc_ret.abs() < 1.0].copy()

price = df["BTC-USD_close"]
print("Rows:", len(df), "Start:", df.index.min(), "End:", df.index.max())

In [ ]:
# Component sleeves

trend_pos = trend_signal(
    df,
    price_column="BTC-USD_close",
    fast_window=TREND_FAST_WINDOW,
    slow_window=TREND_SLOW_WINDOW,
    long_only=TREND_LONG_ONLY,
    leverage_aggressive=TREND_AGGRESSIVE,
    leverage_neutral=TREND_NEUTRAL,
    leverage_defensive=TREND_DEFENSIVE,
)

mining_pos = mining_signal(
    df,
    z_window=MINING_Z_WINDOW,
    entry_z=MINING_ENTRY_Z,
    exit_z=MINING_EXIT_Z,
    use_log_edge=MINING_USE_LOG_EDGE,
)

ou_pos = ou_signal(
    price,
    window=OU_WINDOW,
    entry_z=OU_ENTRY_Z,
    exit_z=OU_EXIT_Z,
    long_short=OU_LONG_SHORT,
)

buy_hold_pos = pd.Series(1.0, index=price.index)
pos_map = {"trend": trend_pos, "mining": mining_pos, "ou": ou_pos}

### Conditional OU selection logic
We start from base **Trend + Mining** with 50/50 weights, then test OU overlays while preserving total weight=1.

For each OU weight `w_ou` in `[0, 0.40]`, Trend and Mining each get `(1 - w_ou)/2`.

If the best OU overlay improves Sharpe vs base, we adopt it. Otherwise we keep base Trend+Mining.

In [ ]:
base_weights = {"trend": 0.50, "mining": 0.50}
base_pos = combine_positions(base_weights, pos_map)
bt_base = run_strategy(price, base_pos)
base_sh = sharpe(bt_base["net_returns"])

search_rows = []
for w_ou in np.linspace(0.0, 0.40, 21):
    w_tm = (1.0 - w_ou) / 2.0
    weights = {"trend": w_tm, "mining": w_tm, "ou": float(w_ou)}
    test_pos = combine_positions(weights, pos_map)
    test_bt = run_strategy(price, test_pos)
    search_rows.append({
        "w_trend": w_tm,
        "w_mining": w_tm,
        "w_ou": float(w_ou),
        "Sharpe": sharpe(test_bt["net_returns"]),
        "bt": test_bt,
        "pos": test_pos,
    })

search_df = pd.DataFrame(search_rows).sort_values("Sharpe", ascending=False)
best_candidate = search_df.iloc[0]

if best_candidate["w_ou"] > 0 and best_candidate["Sharpe"] > base_sh:
    selected_label = "Trend + Mining + OU"
    selected_weights = {
        "trend": float(best_candidate["w_trend"]),
        "mining": float(best_candidate["w_mining"]),
        "ou": float(best_candidate["w_ou"]),
    }
    selected_bt = best_candidate["bt"]
    selected_pos = best_candidate["pos"]
else:
    selected_label = "Trend + Mining"
    selected_weights = {"trend": 0.50, "mining": 0.50, "ou": 0.0}
    selected_bt = bt_base
    selected_pos = base_pos

bt_buy_hold = run_strategy(price, buy_hold_pos)
bt_trend = run_strategy(price, trend_pos)
bt_mining = run_strategy(price, mining_pos)
bt_ou = run_strategy(price, ou_pos)

print(f"Base Trend+Mining Sharpe: {base_sh:.3f}")
print(f"Best candidate Sharpe:    {best_candidate['Sharpe']:.3f}")
print("Selected:", selected_label)
print("Selected weights:", selected_weights)

In [ ]:
# Summary and marginal analysis

def to_row(name: str, bt: dict) -> dict:
    r = bt["net_returns"]
    eq = bt["net_equity"]
    return {
        "Strategy": name,
        "Sharpe": sharpe(r),
        "AnnualReturn": annual_return(r),
        "AnnualVol": annual_vol(r),
        "MaxDD": max_drawdown(eq),
        "TerminalEquity": float(eq.iloc[-1]),
    }

if selected_label == "Trend + Mining + OU":
    bt_with_ou = selected_bt
else:
    # best non-selected OU candidate for comparison
    best_with_ou = search_df[search_df["w_ou"] > 0].iloc[0]
    bt_with_ou = best_with_ou["bt"]

summary = pd.DataFrame([
    to_row("Buy & Hold", bt_buy_hold),
    to_row("Trend", bt_trend),
    to_row("Mining", bt_mining),
    to_row("OU", bt_ou),
    to_row("Trend + Mining (Base)", bt_base),
    to_row("Best Trend + Mining + OU", bt_with_ou),
    to_row(f"Selected ({selected_label})", selected_bt),
]).set_index("Strategy")

marginal = pd.DataFrame({
    "Base Trend+Mining": summary.loc["Trend + Mining (Base)"],
    "Best Trend+Mining+OU": summary.loc["Best Trend + Mining + OU"],
})
marginal["Delta (OU - Base)"] = marginal["Best Trend+Mining+OU"] - marginal["Base Trend+Mining"]

summary.sort_values("Sharpe", ascending=False), marginal

In [ ]:
# Portfolio fit / value-add analysis
# Core proxy: 70% buy-hold + 30% base Trend+Mining

core_returns = 0.70 * bt_buy_hold["net_returns"] + 0.30 * bt_base["net_returns"]
model_returns = selected_bt["net_returns"]

aligned = pd.concat([core_returns, model_returns], axis=1).dropna()
aligned.columns = ["core", "model"]

corr = aligned["core"].corr(aligned["model"])
beta = aligned["model"].cov(aligned["core"]) / aligned["core"].var()
active = aligned["model"] - aligned["core"]

weights = np.linspace(0.0, 0.80, 81)
fit_rows = []
for w in weights:
    combo = (1 - w) * aligned["core"] + w * aligned["model"]
    fit_rows.append({
        "ModelWeight": w,
        "Sharpe": sharpe(combo),
        "AnnualReturn": annual_return(combo),
        "AnnualVol": annual_vol(combo),
    })
fit_df = pd.DataFrame(fit_rows)
best_mix = fit_df.loc[fit_df["Sharpe"].idxmax()]

value_add = {
    "CoreSharpe": sharpe(aligned["core"]),
    "ModelSharpe": sharpe(aligned["model"]),
    "ActiveReturn": annual_return(active),
    "TrackingError": annual_vol(active),
    "InformationRatio": info_ratio(active),
    "CorrCoreModel": float(corr),
    "BetaModelVsCore": float(beta),
    "BestWeight": float(best_mix["ModelWeight"]),
    "BestPortfolioSharpe": float(best_mix["Sharpe"]),
}

pd.Series(value_add)

In [ ]:
# Visual 1: performance curves

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].plot(bt_buy_hold["net_equity"], label="Buy & Hold", alpha=0.8)
ax[0].plot(bt_base["net_equity"], label="Trend+Mining Base", linewidth=2)
ax[0].plot(selected_bt["net_equity"], label=f"Selected ({selected_label})", linewidth=3, linestyle="--")
ax[0].set_title("Net Equity Curves")
ax[0].legend()

ax[1].plot(rolling_sharpe(bt_base["net_returns"], 365), label="Trend+Mining Base")
ax[1].plot(rolling_sharpe(selected_bt["net_returns"], 365), label=f"Selected ({selected_label})")
ax[1].axhline(0, color="black", linewidth=1)
ax[1].set_title("Rolling 365d Sharpe")
ax[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visual 2: marginal value-add + allocation fit

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
bar_series = marginal.loc[["Sharpe", "AnnualReturn", "AnnualVol", "TerminalEquity"], "Delta (OU - Base)"]
bar_colors = ["#2ca02c" if x >= 0 else "#d62728" for x in bar_series.values]
ax[0].bar(bar_series.index, bar_series.values, color=bar_colors)
ax[0].axhline(0, color="black", linewidth=1)
ax[0].set_title("Marginal Impact of OU vs Base")
ax[0].tick_params(axis='x', rotation=25)

ax[1].plot(fit_df["ModelWeight"], fit_df["Sharpe"], linewidth=2)
ax[1].axvline(best_mix["ModelWeight"], linestyle="--", color="red", label=f"Best={best_mix['ModelWeight']:.1%}")
ax[1].set_title("Portfolio Sharpe vs Model Weight")
ax[1].set_xlabel("Weight in Selected Model")
ax[1].set_ylabel("Sharpe")
ax[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visual 3: 3D Sharpe surface over (Trend weight, OU weight)
# Mining weight = 1 - Trend - OU, with feasibility constraint >= 0

grid = np.linspace(0.0, 1.0, 26)
W_T, W_O = np.meshgrid(grid, grid)
S = np.full_like(W_T, np.nan, dtype=float)

for i in range(W_T.shape[0]):
    for j in range(W_T.shape[1]):
        wt = float(W_T[i, j])
        wo = float(W_O[i, j])
        wm = 1.0 - wt - wo
        if wm < 0:
            continue
        pos = combine_positions({"trend": wt, "mining": wm, "ou": wo}, pos_map)
        bt = run_strategy(price, pos)
        S[i, j] = sharpe(bt["net_returns"])

fig = plt.figure(figsize=(11, 7))
ax = fig.add_subplot(111, projection='3d')
mask = np.isfinite(S)
ax.plot_trisurf(W_T[mask], W_O[mask], S[mask], cmap='viridis', linewidth=0.2, antialiased=True)
ax.set_xlabel('Trend Weight')
ax.set_ylabel('OU Weight')
ax.set_zlabel('Net Sharpe')
ax.set_title('3D Sharpe Surface (Mining = 1 - Trend - OU)')
plt.show()